In [ ]:
import json
import networkx as nx
import networkit as nk
import numpy as np
import os
from os.path import join, getsize
from GraphGenerator.preprocessing import dataio

In [ ]:
for graph in data.keys():
    #sub = data[graph].subgraph(next(nx.connected_components(data[graph])))
    with open(f'data/reddit_threads/graph_{graph}.edgelist', 'w') as out_file:
        for edge in data[graph]:
            out_file.write('{} {}'.format(*edge)+'\n')

Analysis if the graph is disconnected

In [ ]:
for root, dirs, files in os.walk('data/ca-condMat'):
    if root.endswith('GraphRNN'):
        continue
    for file in (join(root, name) for name in files):
        if root.endswith('form'):
            continue
        if file.endswith('edgelist'):
            graph = nx.read_edgelist(file)
            fname = file.split('/')[-1].split('.')[0]
            
            mapping = dict(zip(graph, range(len(graph.nodes)+1)))
            graph = nx.relabel_nodes(graph, mapping)
            nx.write_edgelist(graph, f"data/ca-condMat/edgelist_form/{fname}.edgelist", data=False)
            with open(f"data/ca-condMat/col_form/{fname}.col", 'w') as fileheader:
                fileheader.write(f'p edge {len(graph.nodes)} {len(graph.edges)}\n')
                for edge in graph.edges:
                    fileheader.write('e {} {}\n'.format(*tuple(map(lambda x: x+1, edge))))

Convert the pickled networkx graphs list to singular edlist files

In [ ]:
graphs = dataio.load_data('exp/SBMGNN_COLLAB/SBMGNN_top10_2023-Apr-20-11-34-59_506098/top10_to_sbmgnn.graphs')

**Converts the generated Graphs from pickled networkx lists to induvidual files**


In [ ]:

i_offset = 0
dataset = "ER"


def make_folder(path):
    if not os.path.exists(path):
        os.makedirs(path)

[make_folder(path) for path in [f"data/{dataset}/edge_form", f"data/{dataset}/col_form"]]

occs = {}
for root, dirs, files in os.walk(f'exp/{dataset}'):
    if root.endswith(dataset):
        continue
    for file in (join(root, name) for name in files):
        if file.endswith('graphs'):
            graphs = dataio.load_data(file)
            fname = file.split("/")[-1]
            parent_id_s = fname.find('graph_') + 6
            parent_id_e = fname.find('_to')
            parent_id = fname[parent_id_s:parent_id_e]

            if parent_id not in occs:
                occs[parent_id] = 0
            else:
                occs[parent_id] = occs[parent_id] + 1
            
            if graphs[0] is None:
                print(f"Empty graph in {file}")
                continue
            for i, graph in enumerate(graphs):
                largest_cc = graph.subgraph(max(nx.connected_components(graph), key=len))
                largest_cc = nx.convert_node_labels_to_integers(largest_cc, first_label=1)
                #print(type(largest_cc))

                nx.write_edgelist(largest_cc, f"data/{dataset}/edge_form/graph_{parent_id}_{occs[parent_id]}_{i}.edgelist", data=False)
                with open(f"data/{dataset}/col_form/graph_{parent_id}_{occs[parent_id]}_{i}.col", 'w') as fileheader:
                    fileheader.write(f'c Parent graph {dataset} No {parent_id}\n')
                    fileheader.write(f'p edge {len(largest_cc.nodes)} {len(largest_cc.edges)}\n')
                    for edge in largest_cc.edges:
                        #print(largest_cc.edges)
                        #fileheader.write('e {} {}\n'.format(*tuple(map(lambda x: x+1, edge))))
                        fileheader.write('e {} {}\n'.format(*edge))
                

    i_offset += 1

In [ ]:
graph = nk.nxadapter.nx2nk(nx.read_edgelist("data/ca-condMat/ca-CondMat.txt"))
communities = nk.community.detectCommunities(graph)
nk.community.Modularity().getQuality(communities, graph)


In [ ]:
for set_id in communities.getSubsetIds():
    subgraph = nk.graphtools.subgraphFromNodes(graph, communities.getMembers(set_id))
    if subgraph.numberOfNodes() < 100:
        continue
    nk.graphio.writeGraph(subgraph, f"data/ca-condMat/sub_graph_{set_id}.edgelist", nk.Format.EdgeListSpaceZero)


In [ ]:
i = 0
for c in gen:
    nx.write_edgelist(graph.subgraph(c), f"data/ca-condMat/sub_graph_{i}.edgelist")
    i += 1

## Convert the Graphs from TU-DO format to induvidual edgelists

In [ ]:
FOLDER = "DD"
graph = nk.readGraph(f"{FOLDER}_A.txt", nk.Format.EdgeListCommaOne)
graph.removeMultiEdges()
graph.removeSelfLoops()
graph_allocation = np.loadtxt(f"{FOLDER}_graph_indicator.txt", dtype=int)
for graph_id in np.unique(graph_allocation):
    subgraph = nk.graphtools.subgraphFromNodes(graph, (graph_allocation==graph_id).nonzero()[0])
    
    nk.writeGraph(subgraph, f'{FOLDER}_graph_{graph_id}.edgelist', nk.Format.EdgeListSpaceZero)

In [ ]:
import os
os.chdir("/home/martin/devel/uni/thesis/Generator/GraphGenerator/generated_datasets/DD/")

In [ ]:
!ls